In [1]:
from settings import *

df = TRAIN_DF

In [2]:
smiles = df.SMILES
smiles

0                       O=C(Cc1ccc(Cc2ccccc2)cc1)OCC1CO1
1              CCCCCCCCCC(=NO)c1c(O)ccc(-c2ccc(OC)cc2)c1
2                                O=C1C=CC(=O)N1c1ccccc1I
3                        O=C1C=CC(=O)N1c1ccccc1-c1ccccc1
4          O=C1CCC(COC(=O)N2CCC(COc3c(Cl)cc(F)cc3)CC2)N1
                             ...                        
695      O=C(c1ccc(Cl)cc1)C1CCN(C(=O)c2cc(O)cc(Br)c2)CC1
696    O=C(c1ccc(Sc2ccccc2)cc1)C1CCN(C(=O)c2cc(O)c(F)...
697           O=C(N1CCCCC1)n1nnc(Cc2ccc(-c3ccccc3)cc2)n1
698           Cc1ccc(C)n1Cc1ccc(-c2nn(C(=O)N(C)C)nn2)cc1
699                 COc1nn(Cc2cc(OCc3ccccc3)ccc2)c(=O)o1
Name: SMILES, Length: 700, dtype: object

In [3]:
from drugex.data.processing import Standardization


standardizer = Standardization(n_proc=N_PROC)
smiles = standardizer.apply(smiles)

len(smiles)

Standardizing molecules (batch processing):   0%|          | 0/1 [00:00<?, ?it/s]

700

In [4]:
from drugex.data.fragments import FragmentCorpusEncoder
from drugex.data.fragments import GraphFragmentEncoder, FragmentPairsSplitter
from drugex.molecules.converters.fragmenters import Fragmenter
from drugex.data.corpus.vocabulary import VocGraph
from drugex.data.datasets import GraphFragDataSet

encoder = FragmentCorpusEncoder(
    fragmenter=Fragmenter(4, 4, 'brics'),
    encoder=GraphFragmentEncoder(
        VocGraph(n_frags=4)
    ),
    pairs_splitter=FragmentPairsSplitter(0.1), 
    n_proc=N_PROC
)

train = GraphFragDataSet(f"{DATASETS_ENCODED_PATH}/{TARGET_ID}_train.tsv", rewrite=True)
test = GraphFragDataSet(f"{DATASETS_ENCODED_PATH}/{TARGET_ID}_test.tsv", rewrite=True)

encoder.apply(smiles, encodingCollectors=[test, train])

Creating fragment-molecule pairs (batch processing):   0%|          | 0/2 [00:00<?, ?it/s]

Only one retrieved fragment for molecule: C1CN(SSN2CCOCC2)CCO1. Skipping...
Only one retrieved fragment for molecule: S=C(SSC(=S)C1CCCCC1)C1CCCCC1. Skipping...
Only one retrieved fragment for molecule: Cc1coc2c1C(=O)C(=O)c1c-2ccc2c1CCCC2(C)C. Skipping...
Only one retrieved fragment for molecule: CN(C)C(=S)SC(=S)N(C)C. Skipping...
Only one retrieved fragment for molecule: C1CCN(SSN2CCCCC2)CC1. Skipping...
Only one retrieved fragment for molecule: O=[N+]([O-])c1cccc(C(=S)N2CCOCC2)c1. Skipping...
Only one retrieved fragment for molecule: S=C(SSC(=S)N1CCCCCC1)N1CCCCCC1. Skipping...
Only one retrieved fragment for molecule: Oc1ccccc1C(=S)N1CCOCC1. Skipping...
Only one retrieved fragment for molecule: CN(C)C(=S)SSC(=S)N(C)C. Skipping...
Only one retrieved fragment for molecule: S=C(SSC(=S)N1CCOCC1)N1CCOCC1. Skipping...
Only one retrieved fragment for molecule: CC1CCCC(C)N1C(=S)SSC(=S)N1C(C)CCCC1C. Skipping...
Only one retrieved fragment for molecule: S=C(SSC(=S)N1CCCCC1)N1CCCCC1. Skipping...

Encoding fragment-molecule pairs. (batch processing):   0%|          | 0/2 [00:00<?, ?it/s]

Encoding fragment-molecule pairs. (batch processing):   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from drugex.training.models.transform import GraphModel

vocabulary = VocGraph.fromFile(VOCAB_FILE_PR)
pretrained = GraphModel(voc_trg=vocabulary, use_gpus=GPUS_FT)
pretrained.loadStatesFromFile(MODEL_FILE_PR)

In [6]:
from utils import get_timestamp

MODELS_FT_PATH_NOW = f'{MODELS_FT_PATH}/{get_timestamp()}'
MODELS_FT_PATH_NOW

'models/finetuned/Q99685/2023-02-03_12-26-31_027350'

In [7]:
from drugex.training.monitors import FileMonitor


monitor = FileMonitor(
    f"{MODELS_FT_PATH_NOW}/{TARGET_ID}_FT", 
    verbose=True
)

pretrained.fit(
    train.asDataLoader(512),
    test.asDataLoader(512), 
    epochs=N_EPOCHS_FT,
    monitor=monitor
)

print("Finetuning done.")

Fitting model:   0%|          | 0/200 [00:00<?, ?it/s]

Iterating over training batches:   0%|          | 0/10 [00:00<?, ?it/s]

Iterating over validation batches:   0%|          | 0/1 [00:00<?, ?it/s]

Iterating over training batches:   0%|          | 0/10 [00:00<?, ?it/s]

Iterating over validation batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Iterating over training batches:   0%|          | 0/10 [00:00<?, ?it/s]

Iterating over validation batches:   0%|          | 0/1 [00:00<?, ?it/s]

Iterating over training batches:   0%|          | 0/10 [00:00<?, ?it/s]

Iterating over validation batches:   0%|          | 0/1 [00:00<?, ?it/s]

Finetuning done.
